**1- Importation des librairies**

In [1]:
import pandas as pd
import glob
import numpy as np
import sys
import os
import re
from sklearn.ensemble import IsolationForest

**Monter le drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importer un fichier .Py contenant des fonctions utiles

In [3]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [4]:
from function_tools import *

In [5]:
path_files = r"/content/drive/MyDrive/Data_M/bertsom49/"

In [6]:
# récupérer les clusters 
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_file= glob.glob(f"{path_files}*")

In [7]:
list_file

['/content/drive/MyDrive/Data_M/bertsom49/musulman_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/miso_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/gitan_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/chretien_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/capacitiste_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/arabe_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/antisemite_bertsom49',
 '/content/drive/MyDrive/Data_M/bertsom49/africain_bertsom49']

**Extraction des clusters pertinent**

In [8]:
def make_liste_pertinent(liste):
  liste_brut=[]
  liste_vec=[]
  liste_info=[]
  for file in liste:
    liste_brut.append(glob.glob(file+"/*raw*"))
    liste_vec.append(glob.glob(file+"/*encoded*"))
    liste_info.append(glob.glob(file+"/*info*"))
  return liste_vec, liste_brut,liste_info

In [9]:
def extract_pourcentage(filename):
  pourcentage_haineux = []
  with open(filename, "r") as f:
    lines = f.read().split('\n\n')
  for line in lines[:-1]: 
    hper = re.findall('(\d*.\d*)% de', line)
    hper = float(hper[0]) if len(hper) > 0 else 0
    pourcentage_haineux.append(hper)
  return pourcentage_haineux

In [10]:
liste_vec, liste_brut, liste_info =make_liste_pertinent(list_file)

In [11]:
for liste in liste_vec:
  liste.sort(key=extr_num2)

In [ ]:
liste_vec

In [ ]:
liste_brut

In [ ]:
liste_info

[['/content/drive/MyDrive/Data_M/bertsom49/musulman_bertsom49/musulman_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/miso_bertsom49/miso_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/gitan_bertsom49/gitan_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/chretien_bertsom49/chretien_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/capacitiste_bertsom49/capacitiste_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/arabe_bertsom49/arabe_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/antisemite_bertsom49/antisemite_bertsom49_clusters_info.txt'],
 ['/content/drive/MyDrive/Data_M/bertsom49/africain_bertsom49/africain_bertsom49_clusters_info.txt']]

In [12]:
def extract_all(liste):
  taux=[]
  for el in liste:
    taux.append(extract_pourcentage(el[0]))
  return taux

In [13]:
pourcentage_haineux= extract_all(liste_info)

In [ ]:
pourcentage_haineux

In [15]:
def filtre_cluster_pertinent(cont):
  pertinent_cluster=[]
  for i in range(len(cont)):
    if cont[i]!= 0 and cont[i] < 20 or cont[i] > 80 :
      pertinent_cluster.append(i)
  return pertinent_cluster

In [16]:
def filter_all(liste):
  clusters=[]
  for el in liste:
    clusters.append(filtre_cluster_pertinent(el))
  return clusters    

In [17]:
pertinent_cluster=filter_all(pourcentage_haineux)

In [ ]:
len(pertinent_cluster)

8

**2- Lire le contenu des clusters**

In [18]:
pertinent_cluster[0]

[8, 9, 10, 15, 17, 21, 24, 31, 38, 46]

In [19]:
def read_cluster_df(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(pd.DataFrame(file.readlines(), columns=['tweet']))
  return liste

In [22]:
def matrice_df(liste):
  cluster_km=[]
  for file in liste:
    file.sort(key=extr_num2)
    cluster_km.append(read_cluster_df(file))
  return cluster_km

In [23]:
%%time
brut_cluster= matrice_df(liste_brut)

CPU times: user 591 ms, sys: 134 ms, total: 726 ms
Wall time: 52.5 s


In [ ]:
len(brut_cluster)

8

In [24]:
brut_cluster[1][0]

,tweet
0,content\n
1,Tou bué fart@\n
2,Nazi Barbie. Putin biatch\n
3,common sense...\n
4,Tf he do? 😭😂\n
...,...
95,Comments\n
96,Salope ultime niveau 100. Boss imbattable\n
97,Caleta Car ta mère la grosse pute\n
98,allume fils de pute\n


**4- Mettre les vecteurs dans une liste**

In [ ]:
liste_vec

In [26]:
def load_array(liste):
  liste_vec=[]
  for i in range(len(liste)):
    liste_vec.append(np.load(liste[i]))
  return liste_vec

In [27]:
liste_af=load_array(liste_vec[7])

In [28]:
liste_af[14].shape

(188, 768)

In [33]:
def matrice_array(liste):
  liste_vec=[]
  for i in range(len(liste)):
      liste_vec.append(load_array(liste[i]))
  return liste_vec

In [34]:
liste_npy=matrice_array(liste_vec)

In [ ]:
liste_npy[7][14].shape

(188, 768)

In [ ]:
liste_africain=liste_npy[7]

In [ ]:
liste_africain[14].shape

(188, 768)

**5- Entrainement avec Isolation Forest**

**A- Variante contamination fixe**

In [35]:
def train_if (dataset):
  models=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      if_model=IsolationForest(n_jobs=-1, random_state=42, contamination=0.15)
      dataset[i][n].shape
      models[i].append(if_model.fit(dataset[i][n]))
  return models

In [36]:
# Entrainer le modèle
%%time
models = train_if(liste_npy)

CPU times: user 2min 1s, sys: 7.8 s, total: 2min 9s
Wall time: 2min 5s


In [37]:
def pred(dataset,model):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      preds[i].append(model[i][n].predict(dataset[i][n]))
  return preds

In [38]:
def pred_df(dataset,model, data):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      data[i][n]['pred']=model[i][n].predict(dataset[i][n])
  

In [39]:
pred_df(liste_npy, models, brut_cluster)

In [ ]:
brut_cluster[1][0]

,tweet,pred
0,content\n,-1
1,Tou bué fart@\n,1
2,Nazi Barbie. Putin biatch\n,1
3,common sense...\n,1
4,Tf he do? 😭😂\n,1
...,...,...
95,Comments\n,1
96,Salope ultime niveau 100. Boss imbattable\n,-1
97,Caleta Car ta mère la grosse pute\n,1
98,allume fils de pute\n,1


In [40]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def separate_outliers(liste_df, pertinent):
  clusters_clean=[]
  for i,n in enumerate(pertinent):
    clusters_clean.append(liste_df[n]['tweet'].loc[liste_df[n]['pred']==1])
    print("cluster"+str(n)+":"+"\n")
    print(liste_df[n]['pred'].value_counts())
    print("\n")
  return clusters_clean

In [41]:
def separate_matrice(liste, pertinent_cluster):
  liste_clean=[]
  for i in range(len(liste)):
    liste_clean.append(separate_outliers(liste[i], pertinent_cluster[i]))
  return liste_clean

In [ ]:
liste_clean= separate_matrice(brut_cluster, pertinent_cluster)

In [43]:
for i in range(len(pertinent_cluster)):
  print(len(pertinent_cluster[i]))


10
27
13
14
21
19
17
15


In [ ]:
for i in range(len(liste_clean)):
  print(len(liste_clean[i]))

10
27
13
14
21
19
17
15


In [44]:
def save_cluster(liste, file, repo, pertinent):
  for i,n in enumerate(pertinent):
    with open(f"drive/MyDrive/"+repo+file+f"_{n}", 'w') as f:
      for line in liste[i]:
        f.write(line)

In [47]:
liste=["mus","miso","gitan","chret","capa","arabe","antis","afro"]

In [48]:
def loop_outliers(data, liste, chaine, pertinent):
  for i in range(len(data)):
    save_cluster(data[i],liste[i], chaine,pertinent[i])

In [49]:
loop_outliers(liste_clean, liste, "clean_bert_som49/", pertinent_cluster)